In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as tkr
import pickle
import urllib2
import json
import datetime
import csv
import time
import re
import string
import sqlite3

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [23]:
# the following facebook page post scraper was taken from 
# https://drive.google.com/file/d/0Bw1LIIbSl0xuRTNCZElUa3U1b1U/view

# study period from August 26, 2016 to January 20, 2017 (includes 73 between election day)
app_id = "1239416149446116"
app_secret = "b75da335f4fab048c1564a066b4848ba" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#access_token = raw_input("Please Paste Your Access Token:")

def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)

            print "Error for URL %s: %s" % (url, datetime.datetime.now())
            print "Retrying."

    return response.read()

# Needed to write tricky unicode correctly to csv
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22,
                            0xa0:0x20 }).encode('utf-8')

def getFacebookPageFeedData(page_id, access_token, num_statuses):

    base = "https://graph.facebook.com/v2.6"
    node = "/%s/posts" % page_id 
    fields = "/?fields=message,link,permalink_url,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)"
    parameters = "&until=2017-01-21&since=2016-08-25&limit=%s&access_token=%s" % (num_statuses, access_token)
    
    url = base + node + fields + parameters

    # retrieve data
    
    data = json.loads(request_until_succeed(url))
    return data

def getReactionsForStatus(status_id, access_token):
    base = "https://graph.facebook.com/v2.6"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&until=2017-01-21&since=2016-08-25&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))
     
    return data

def processFacebookPageFeedStatus(status, access_token):

    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else \
            unicode_normalize(status['message'])
    link_name = '' if 'name' not in status.keys() else \
            unicode_normalize(status['name'])
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else \
            unicode_normalize(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_normalize(status['permalink_url'])
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.

    status_published = datetime.datetime.strptime(
            status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + \
            datetime.timedelta(hours=-8) # PST
    status_published = status_published.strftime(
            '%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    # Nested items require chaining dictionary keys.

    num_reactions = 0 if 'reactions' not in status else \
            status['reactions']['summary']['total_count']
    num_comments = 0 if 'comments' not in status else \
            status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status else status['shares']['count']

    # Counts of each reaction separately; good for sentiment
    # Only check for reactions if past date of implementation:
    # http://newsroom.fb.com/news/2016/02/reactions-now-available-globally/

    reactions = getReactionsForStatus(status_id, access_token) if \
            status_published > '2016-02-24 00:00:00' else {}

    num_likes = 0 if 'like' not in reactions else \
            reactions['like']['summary']['total_count']

    # Special case: Set number of Likes to Number of reactions for pre-reaction
    # statuses

    num_likes = num_reactions if status_published < '2016-02-24 00:00:00' \
            else num_likes

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']

    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)

    # Return a tuple of all processed data

    return (status_id, status_message, link_name, status_type, status_link, status_permalink_url,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)


def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type",
                    "status_link", "permalink_url", "status_published", "num_reactions", 
                    "num_comments", "num_shares", "num_likes", "num_loves", 
                    "num_wows", "num_hahas", "num_sads", "num_angrys"])

        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()

        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)

        statuses = getFacebookPageFeedData(page_id, access_token, 100)

        while has_next_page:
            for status in statuses['data']:
                # Ensure it is a status with the expected metadata
                if 'reactions' in status:
                    w.writerow(processFacebookPageFeedStatus(status,
                        access_token))

                # output progress occasionally to make sure code is not
                # stalling
                num_processed += 1
                if num_processed % 100 == 0:
                    print "%s Statuses Processed: %s" % \
                        (num_processed, datetime.datetime.now())

            # if there is no next page, we're done.
            if ('paging' in statuses.keys()):
                statuses = json.loads(request_until_succeed(
                                        statuses['paging']['next']))
            else:
                has_next_page = False


        print "\nDone!\n%s Statuses Processed in %s" % \
                (num_processed, datetime.datetime.now() - scrape_starttime)


In [43]:
# mainstream sources trusted more than not according to Pew study

mainstream=["nytimes", "wsj", "ABCNews", "CBSNews", "cnn","usatoday", "washingtonpost",
           "msnbc","newyorker","yahoonews","FoxNews", "NPR", "newshour"]
print sorted(mainstream)
print len(mainstream)
# for news in mainstream:
#     page_id=news
#     scrapeFacebookPageFeedStatus(page_id, access_token)

['ABCNews', 'CBSNews', 'FoxNews', 'NPR', 'cnn', 'msnbc', 'newshour', 'newyorker', 'nytimes', 'usatoday', 'washingtonpost', 'wsj', 'yahoonews']
13


In [39]:
# doesn't work: 'conservativegroup', 
# done:  
fake=['worldrumor',
     'newswatch28','FreshNews000','TheRightists','subjectpolitics', 'weeklyworldnews', 'USPoliticsLive',
     'DcGazette','FreedomDailyNews','EmpireNewsNet','Newslo1','NewsWithViews','supremepatriot',
     'StormCloudsGathering', 'DHeadlines', 'newsexaminer.net','24Newsflash','LastGreatStand','eutopiabuzz',
     'conservativeinfidel','ThreePercenterNation','TheUndergroundWorldNews','RHobbusJD',
     'downtrendcom','intrendtoday','ClashDaily','worldstoriestoday','TheAdoboChronicles','beforeitsnewscom', 
      'DonaldTrumpNews.Co', 'hmakow', 'pakalertpress','usasupreme','rilenews', 'RickRWells', 'usanewsoftheday',
      'yesiamright', 'proudcons', 'conservativebyte','politicalsitenews', 'prntly', 'OnlineConservativePress']
print sorted(fake)
print len(fake)

# for news in fake:
#     page_id=news
#     scrapeFacebookPageFeedStatus(page_id, access_token)

['24Newsflash', 'ClashDaily', 'DHeadlines', 'DcGazette', 'DonaldTrumpNews.Co', 'EmpireNewsNet', 'FreedomDailyNews', 'FreshNews000', 'LastGreatStand', 'NewsWithViews', 'Newslo1', 'OnlineConservativePress', 'RHobbusJD', 'RickRWells', 'StormCloudsGathering', 'TheAdoboChronicles', 'TheRightists', 'TheUndergroundWorldNews', 'ThreePercenterNation', 'USPoliticsLive', 'beforeitsnewscom', 'conservativebyte', 'conservativeinfidel', 'downtrendcom', 'eutopiabuzz', 'hmakow', 'intrendtoday', 'newsexaminer.net', 'newswatch28', 'pakalertpress', 'politicalsitenews', 'prntly', 'proudcons', 'rilenews', 'subjectpolitics', 'supremepatriot', 'usanewsoftheday', 'usasupreme', 'weeklyworldnews', 'worldrumor', 'worldstoriestoday', 'yesiamright']
42


In [46]:
# Can not use: 'WorldTruthTV','zootfeednews',
conspiracy=['pamelageller','fprnradio','americanfreepress','LibertyTalkFM','TheMindUnleashed','adeptoerperfectus', 
            'Gaia','conservativerefocusgroup','CanadaFreePress','concisepolitics','FourWinds10','ThePoliticalInsider',
            'DisclosureMedia','TheSilverDoctors','skeptiko.us','21WIRE.TV','atsnews','activistpost','TheCorbettReport',
            'EyeOpeningInfo','freedomoutpost','GlobalResearchCRG','godlikeproductions','govtslaves','HumansAreFree',
            'investmentresearchdynamics','intelligencehub','NewsTargetOfficial','NowTheEndBegins','theprepperwebsite',
            'SECRETSofTheFED','thedailysheeple','theeventhandbook','TruthBroadcastNetworkcom',
            'truth.frequency.radio','ufoholics','zerohedge1','SilverCoinInvestor','LibertyMovementRadio',
            'BradleeDeanSOL','TheLibertyBeaconProject','AwarenessAct']
print sorted(conspiracy)
print len(conspiracy)

# for news in conspiracy:
#     page_id=news
#     scrapeFacebookPageFeedStatus(page_id, access_token)

['21WIRE.TV', 'AwarenessAct', 'BradleeDeanSOL', 'CanadaFreePress', 'DisclosureMedia', 'EyeOpeningInfo', 'FourWinds10', 'Gaia', 'GlobalResearchCRG', 'HumansAreFree', 'LibertyMovementRadio', 'LibertyTalkFM', 'NewsTargetOfficial', 'NowTheEndBegins', 'SECRETSofTheFED', 'SilverCoinInvestor', 'TheCorbettReport', 'TheLibertyBeaconProject', 'TheMindUnleashed', 'ThePoliticalInsider', 'TheSilverDoctors', 'TruthBroadcastNetworkcom', 'activistpost', 'adeptoerperfectus', 'americanfreepress', 'atsnews', 'concisepolitics', 'conservativerefocusgroup', 'fprnradio', 'freedomoutpost', 'godlikeproductions', 'govtslaves', 'intelligencehub', 'investmentresearchdynamics', 'pamelageller', 'skeptiko.us', 'thedailysheeple', 'theeventhandbook', 'theprepperwebsite', 'truth.frequency.radio', 'ufoholics', 'zerohedge1']
42


In [45]:
# Can not use: ncscooper, 'NationalReportOnline',
satire=['TheHolyObserver','CallTheCopsNews','NewsToad',
        'associatedmediacoverage','LiberalBiasBlog','dailycurrant','theineptowl','betootaadvocate','clickhole','disclosetv', 'elmundotoday', 'Gomerblog','NewsBiscuit',
       'NewsThump','Reductress','TheOnion','theunrealpage','WhispersNews','goingwunderground','FreeWoodPost',
       'thetimesoftheworld','worldnewsdailyreport','BeehiveBugle','thedandygoat','RockCityTimes',
       'CreamBmp','TheMadisonMisnomer','liberaldarkness','abril.uno.satire','TheStatelyHarold','SportsPickle',
       'TheSkunkPage','stuppidcom','witscience','SatiraTribune','HumorTimes','TheNewsNerd','blastingusa',
       'elkoshary','islamicanews','BurrardStreetJournal','thehardtimesnews','TheBeaverton','FridayMash',
       'TheCelebtricity']
print sorted(satire)
print len(satire)

# for news in satire:
#     page_id=news
#     scrapeFacebookPageFeedStatus(page_id, access_token)

['BeehiveBugle', 'BurrardStreetJournal', 'CallTheCopsNews', 'CreamBmp', 'FreeWoodPost', 'FridayMash', 'Gomerblog', 'HumorTimes', 'LiberalBiasBlog', 'NewsBiscuit', 'NewsThump', 'NewsToad', 'Reductress', 'RockCityTimes', 'SatiraTribune', 'SportsPickle', 'TheBeaverton', 'TheCelebtricity', 'TheHolyObserver', 'TheMadisonMisnomer', 'TheNewsNerd', 'TheOnion', 'TheSkunkPage', 'TheStatelyHarold', 'WhispersNews', 'abril.uno.satire', 'associatedmediacoverage', 'betootaadvocate', 'blastingusa', 'clickhole', 'dailycurrant', 'disclosetv', 'elkoshary', 'elmundotoday', 'goingwunderground', 'islamicanews', 'liberaldarkness', 'stuppidcom', 'thedandygoat', 'thehardtimesnews', 'theineptowl', 'thetimesoftheworld', 'theunrealpage', 'witscience', 'worldnewsdailyreport']
45


In [ ]:
# Further analysis from main notebook, separated to conduct multiple analyses simulaneously

In [8]:
sqlite_db = '/Users/lukejarmbruster/Desktop/DSI-SF-4-larmbruster/projects/capstone/facebook_news.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [9]:
fb_news_total=pd.read_sql('SELECT * FROM fbook_news', con=conn)

In [10]:
conn.close()

In [11]:
# convert object columns from unicode to string
def decoder(x):
    if not x:
        return ''
    else:
        x_ = ''.join([ch for ch in x if ch in string.printable])
        return str(x_.decode('ascii'))
object_types=fb_news_total.columns[fb_news_total.dtypes=='object']
for i in object_types:
    fb_news_total[i] = fb_news_total[i].map(decoder)

In [12]:
fb_news_total.status_published=pd.to_datetime(fb_news_total.status_published)

In [13]:
# study period from August 26, 2016 [including] to January 19, 2017 [including ](includes 73 
# before and after election day)

mask=((fb_news_total['status_published'] >= '2016-08-26') & (fb_news_total['status_published'] < '2017-01-20')) 
fb_news_total=fb_news_total.loc[mask]
fb_news_total['election_day']='same'
fb_news_total.ix[fb_news_total['status_published']<'2016-11-07','election_day']='before'
fb_news_total.ix[fb_news_total['status_published']>='2016-11-08','election_day']='after'
fb_news_total.reset_index(inplace=True, drop=True)

#fb_news_total.ix[fb_news_total['election_day']=='same','status_published'].sort_values()
del mask

In [14]:
# add time columns
fb_news_total['day_of_week']=fb_news_total.status_published.dt.weekday_name
fb_news_total['year']=fb_news_total.status_published.dt.year
fb_news_total['month']=fb_news_total.status_published.dt.month
fb_news_total['day']=fb_news_total.status_published.dt.day
fb_news_total['hour']=fb_news_total.status_published.dt.hour

In [15]:
# investigate repeats in status_id, status_message, link_name, status_link, permalink_url
# removed duplicate posts
mask=((fb_news_total.duplicated(['status_id'])) & (fb_news_total.duplicated(['status_message'])) & 
      (fb_news_total.duplicated(['link_name'])) & (fb_news_total.duplicated(['status_link'])) &
      (fb_news_total.duplicated(['permalink_url'])) & (fb_news_total.duplicated(['status_published']))
     & (fb_news_total.duplicated(['num_reactions'])))

print sum(mask)
print sum(~mask)

# remove the rows with that repeat six of the same columns elements as another row
fb_news_total=fb_news_total[~mask]
fb_news_total.reset_index(inplace=True, drop=True)
del mask

99
274618


In [ ]:
fb_news_total['all_interactions']=fb_news_total['num_reactions']+fb_news_total['num_shares']+fb_news_total['num_comments']

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy import en

In [17]:
addstop=['bz', 'rgn', 'ly','000','li','ws','ow','pos','\n','"','2g8inr1','20170113','2evy5ku','abcn','clickhole']
for i in addstop:
    en.STOPWORDS.add(i)

In [18]:
fb_news_total['status_message'][5]

'Our photographer Doug Mills has taken "hundreds of thousands" of photos of Barack Obama. "He wears his emotions on his sleeve," he said. "You can see when he\'s tense. Or jovial. The passion is there."'

In [19]:
# clean text
#len(fb_news_total['status_message'])

In [ ]:
for i in range(len(fb_news_total['status_message'])):
    fb_news_total.ix[i,'status_message']=re.sub(r'[-a-zA-Z0-9@:%_\+.~#?&//=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)?', ' ', fb_news_total.ix[i,'status_message'])
    fb_news_total.ix[i,'status_message']=re.sub(r'\/', ' ', fb_news_total.ix[i,'status_message'])
    fb_news_total.ix[i,'status_message']=re.sub('/n', ' ', fb_news_total.ix[i,'status_message'])

In [ ]:
analyzer = SentimentIntensityAnalyzer()

fb_news_total['sentiment_comp_score']=None
for i in range(len(fb_news_total['status_message'])):
    sentences=sent_tokenize(fb_news_total.ix[i,'status_message'])
    templist=[]
    for j in sentences:
        try:
            single_score = analyzer.polarity_scores(j)
            templist.append(single_score['compound'])
            print j
            print single_score
            print ""
        except:
            # writing to a file with pickle
            # 'w' is for write, note that it will rewrite any existing obj
            if len(templist)==0:
                fb_news_total.ix[i,'sentiment_comp_score']=None
            else:
                fb_news_total.ix[i,'sentiment_comp_score']=np.mean(templist)
            with open('/Users/lukejarmbruster/Desktop/DSI-SF-4-larmbruster/projects/capstone/sentiment_output_clean.pkl', 'w') as f:
                pickle.dump(fb_news_total, f)
            pass
    if len(templist)==0:
        fb_news_total.ix[i,'sentiment_comp_score']=None
    else:
        fb_news_total.ix[i,'sentiment_comp_score']=np.mean(templist)

with open('/Users/lukejarmbruster/Desktop/DSI-SF-4-larmbruster/projects/capstone/sentiment_output_clean.pkl', 'w') as f:
    pickle.dump(fb_news_total, f)